In [1]:
import pandas as pd
import numpy as np
import pickle
from LightGBMwithBayesOpt.LightGBMwithBayesOpt import LightGBMwithBayesOpt, LightGBMEvalutor

In [2]:
### data for 0 plan 
### all three category columns of channel, fee_prod_id and mbr_card_gr are already label-encoded
### all numeric columns are already standardization-scaled

with open('train_set_0304.pkl', 'rb') as f:
    train_X, train_y = pickle.load(f)
with open('test_set_0405.pkl', 'rb') as f:
    test_X, test_y = pickle.load(f)

In [3]:
train_X.head()

,channel,fee_prod_id,mbr_card_gr,additional_svc_allcare_scrb_type_standard,additional_svc_ansim_option_scrb_type_standard,additional_svc_bugs_scrb_type_standard,additional_svc_flo_scrb_type_standard,additional_svc_melon_scrb_type_standard,additional_svc_oksusu_scrb_type_standard,additional_svc_pooq_scrb_type_standard,...,overage_bf0_standard,overage_bf1_standard,overage_bf2_standard,real_avg_arpu_standard,rem_allot_cnt_standard,roaming_cnt_m3_standard,simpl_ref_cnsl_cnt_m3_standard,smart_watch_cnt_standard,svc_scrb_period_standard,tablet_cnt_standard
252483,1,247,2,-0.019464,-0.290523,-0.114694,-0.11104,-0.075765,2.317187,0.0,...,-0.202224,-0.191739,-0.201029,-0.232049,-0.920419,-0.117019,-0.338241,-0.057639,3.161220,-0.031772
252484,2,171,2,-0.019464,-0.290523,-0.114694,-0.11104,-0.075765,0.528471,0.0,...,-0.202224,-0.191739,-0.201029,-0.027538,-0.498600,-0.117019,0.233286,-0.057639,2.691778,-0.031772
252485,3,204,4,-0.019464,-0.290523,-0.114694,-0.11104,-0.075765,0.528471,0.0,...,-0.202224,-0.191739,-0.201029,0.198199,-0.393145,-0.117019,-0.338241,-0.057639,2.626016,-0.031772
252486,2,168,4,-0.019464,-0.290523,-0.114694,-0.11104,-0.075765,0.528471,0.0,...,-0.202224,-0.191739,-0.201029,-0.587029,1.399585,-0.117019,0.233286,-0.057639,2.314404,-0.031772
252487,2,141,0,-0.019464,-0.290523,-0.114694,-0.11104,-0.075765,0.528471,0.0,...,-0.202224,-0.191739,-0.201029,-1.172890,-0.920419,-0.117019,1.376341,-0.057639,2.307322,-0.031772


In [4]:
pd.value_counts(train_y)

0Large      11222
Special      5237
0Medium      4969
Essence      3326
0Small       3275
Safe2.5G     1242
Max           419
Safe4G        419
Name: price_plan, dtype: int64

In [5]:
not_range_params = {
    'boosting': ['gbdt', 'dart'],
    'objective': ['multiclass', 'multiclassova'],
    'is_unbalance': [False, True]
}
rangeable_params = {
    'num_leaves': (20, 100),
    'max_depth': (3, 10),
    'learning_rate': (0.01, 0.5),
    'num_iterations': (100, 300),
    'bagging_fraction': (0.8, 1),
    'bagging_freq': (2, 10),
    'feature_fraction': (0.6, 1)
}
int_params = ['num_leaves', 'max_depth', 'num_iterations', 'bagging_freq']

In [7]:
#### multiclassification modeling for 0Small, 0Medium and 0Large 
#### while taking the category columns as category
#### 
multiclass_tuner = LightGBMwithBayesOpt(train_X, train_y, 3, rangeable_params, int_params, 5, ['0Small', '0Medium', '0Large'], 
                                        not_range_params, categorical_feature=['channel', 'fee_prod_id', 'mbr_card_gr'], 
                                        pred_modes=['prob_first'], metrics=['accuracy', 'mcc'])
multiclass_tuner.do_parameter_tuning()

3275
4969
11222
|   iter    |  target   | baggin... | baggin... | boosting  | featur... | is_unb... | learni... | max_depth | num_it... | num_le... | objective |
-------------------------------------------------------------------------------------------------------------------------------------------------
{'bagging_fraction': 0.9384113947910206, 'bagging_freq': 2, 'boosting': 'gbdt', 'feature_fraction': 0.761061672706952, 'is_unbalance': False, 'learning_rate': 0.1918109842621281, 'max_depth': 3, 'num_iterations': 103, 'num_leaves': 80, 'objective': 'multiclassova', 'num_threads': 20, 'num_class': 3}
34401


/home/dmig/py_libs/lib/python3.6/site-packages/lightgbm/engine.py:118: UserWarning: Found `num_iterations` in params. Will use it instead of argument
  warnings.warn("Found `{}` in params. Will use it instead of argument".format(alias))
/home/dmig/py_libs/lib/python3.6/site-packages/lightgbm/basic.py:1209: UserWarning: categorical_feature in Dataset is overridden.
New categorical_feature is ['channel', 'fee_prod_id', 'mbr_card_gr']
  'New categorical_feature is {}'.format(sorted(list(categorical_feature))))


   accuracy_prob_first  mcc_prob_first
0             0.130591        0.158809
1             0.130301        0.158658
2             0.130650        0.159450
mean: 0.14474310698594162
|  1        |  0.1447   |  0.9384   |  2.356    |  0.1911   |  0.7611   |  0.4963   |  0.1918   |  3.193    |  103.4    |  79.57    |  0.7314   |
{'bagging_fraction': 0.9541908798524932, 'bagging_freq': 6, 'boosting': 'gbdt', 'feature_fraction': 0.749531929546545, 'is_unbalance': False, 'learning_rate': 0.40194253141400144, 'max_depth': 8, 'num_iterations': 224, 'num_leaves': 72, 'objective': 'multiclassova', 'num_threads': 20, 'num_class': 3}
34401


/home/dmig/py_libs/lib/python3.6/site-packages/lightgbm/engine.py:118: UserWarning: Found `num_iterations` in params. Will use it instead of argument
  warnings.warn("Found `{}` in params. Will use it instead of argument".format(alias))
/home/dmig/py_libs/lib/python3.6/site-packages/lightgbm/basic.py:1209: UserWarning: categorical_feature in Dataset is overridden.
New categorical_feature is ['channel', 'fee_prod_id', 'mbr_card_gr']
  'New categorical_feature is {}'.format(sorted(list(categorical_feature))))


   accuracy_prob_first  mcc_prob_first
0             0.129735        0.155572
1             0.130130        0.156591
2             0.131090        0.159134
mean: 0.14370849715736383
|  2        |  0.1437   |  0.9542   |  5.637    |  0.09156  |  0.7495   |  0.3053   |  0.4019   |  7.794    |  223.9    |  71.58    |  0.7723   |
{'bagging_fraction': 0.9972688490179249, 'bagging_freq': 8, 'boosting': 'dart', 'feature_fraction': 0.6688562210159074, 'is_unbalance': True, 'learning_rate': 0.16362386763639233, 'max_depth': 8, 'num_iterations': 223, 'num_leaves': 51, 'objective': 'multiclass', 'num_threads': 20, 'num_class': 3}
34401


...

   accuracy_prob_first  mcc_prob_first
0             0.129613        0.155698
1             0.130130        0.156858
2             0.130796        0.158483
mean: 0.14359636048321006
|  13       |  0.1436   |  0.9316   |  3.017    |  0.07748  |  0.8139   |  0.8135   |  0.1775   |  4.696    |  299.9    |  99.63    |  0.1408   |
{'bagging_fraction': 0.9417847991903163, 'bagging_freq': 4, 'boosting': 'gbdt', 'feature_fraction': 0.6021678279297499, 'is_unbalance': False, 'learning_rate': 0.25495887501145537, 'max_depth': 9, 'num_iterations': 299, 'num_leaves': 100, 'objective': 'multiclassova', 'num_threads': 20, 'num_class': 3}
34401


/home/dmig/py_libs/lib/python3.6/site-packages/lightgbm/engine.py:118: UserWarning: Found `num_iterations` in params. Will use it instead of argument
  warnings.warn("Found `{}` in params. Will use it instead of argument".format(alias))
/home/dmig/py_libs/lib/python3.6/site-packages/lightgbm/basic.py:1209: UserWarning: categorical_feature in Dataset is overridden.
New categorical_feature is ['channel', 'fee_prod_id', 'mbr_card_gr']
  'New categorical_feature is {}'.format(sorted(list(categorical_feature))))


   accuracy_prob_first  mcc_prob_first
0             0.130078        0.156497
1             0.130570        0.157765
2             0.131285        0.159518
mean: 0.14428531466145103
|  14       |  0.1443   |  0.9418   |  3.631    |  0.4763   |  0.6022   |  0.04536  |  0.255    |  8.561    |  299.3    |  99.62    |  0.8056   |
{'bagging_fraction': 0.9194554605111841, 'bagging_freq': 3, 'boosting': 'dart', 'feature_fraction': 0.7335728796099623, 'is_unbalance': True, 'learning_rate': 0.4872015757926172, 'max_depth': 4, 'num_iterations': 300, 'num_leaves': 100, 'objective': 'multiclassova', 'num_threads': 20, 'num_class': 3}
34401


/home/dmig/py_libs/lib/python3.6/site-packages/lightgbm/engine.py:118: UserWarning: Found `num_iterations` in params. Will use it instead of argument
  warnings.warn("Found `{}` in params. Will use it instead of argument".format(alias))
/home/dmig/py_libs/lib/python3.6/site-packages/lightgbm/basic.py:1209: UserWarning: categorical_feature in Dataset is overridden.
New categorical_feature is ['channel', 'fee_prod_id', 'mbr_card_gr']
  'New categorical_feature is {}'.format(sorted(list(categorical_feature))))


   accuracy_prob_first  mcc_prob_first
0             0.129760        0.159780
1             0.129616        0.159365
2             0.130209        0.160894
mean: 0.14493734279191772
|  15       |  0.1449   |  0.9195   |  2.971    |  0.9695   |  0.7336   |  0.8142   |  0.4872   |  3.704    |  299.9    |  99.68    |  0.5225   |


In [12]:
### train data with the best parameter
multiclass_model = multiclass_tuner.do_train_with_given_parameter(multiclass_tuner.best_parameter, multiclass_tuner.data_set)

In [14]:
multiclass_model.params

{'bagging_fraction': 0.9823658146002981,
 'bagging_freq': 2,
 'boosting': 'dart',
 'feature_fraction': 0.844557193424584,
 'is_unbalance': True,
 'learning_rate': 0.10260227101636475,
 'max_depth': 10,
 'num_class': 3,
 'num_leaves': 100,
 'num_threads': 20,
 'objective': 'multiclassova'}

In [24]:
### evaluate the model with the test set
test_y.loc[test_y.isnull()] = '-1'
evl = LightGBMEvalutor(multiclass_model, test_X, ['0Small', '0Medium', '0Large'], test_y)
scores = evl.score_predict()
raw_scores = evl.score_predict(raw=True)
prob_first = evl.label_predict(scores, mode='prob_first')
rank_first = evl.label_predict(raw_scores, mode='rank_first')
acc_mcc_list = list(map(lambda y: list(map(lambda x: evl.measure_pred_with_true(y, mode=x), ['accuracy', 'mcc'])), [prob_first, rank_first]))

In [25]:
scores.head()

,0Small,0Medium,0Large
0,0.000019,0.000138,0.999811
1,0.000037,0.000592,0.999793
2,0.000076,0.431975,0.268015
3,0.000053,0.000116,0.999815
4,0.000031,0.573192,0.471622


In [26]:
raw_scores.head()

,0Small,0Medium,0Large
0,-10.882721,-8.886824,8.573075
1,-10.194631,-7.431277,8.482677
2,-9.481941,-0.273798,-1.004715
3,-9.837602,-9.063112,8.595174
4,-10.394949,0.294886,-0.113636


In [27]:
prob_first[:5]

['0Large', '0Large', '0Medium', '0Large', '0Medium']

In [28]:
rank_first[:5]

['0Large', '0Large', '0Small', '0Large', '0Medium']

In [30]:
## accuracy_prob_first, mcc_prob_first
## accuracy_rank_first, mcc_rank_first
acc_mcc_list

[[0.2947482208810017, 0.226415448288777],
 [0.3079452301594451, 0.2695706723822431]]